# question 2

In [1]:
from os import path
import os
import pandas as pd
import numpy as np
import gurobipy as gb
from gurobipy import GRB
from gurobipy import quicksum
import math

In [2]:
os.getcwd()

'/Users/eason/Desktop/OperationReseach/Case2'

In [3]:
work_dir = os.getcwd()
data_path = path.join(work_dir, 'data')
data_file = path.join(data_path, 's5.xlsx')

In [4]:
Demand_df = pd.read_excel(data_file, sheet_name='Demand', index_col='Product')
Init_df = pd.read_excel(data_file, sheet_name='Initial inventory', index_col='Product')
ShippingCost_df = pd.read_excel(data_file, sheet_name='Shipping cost', index_col='Product')
InTransit_df = pd.read_excel(data_file, sheet_name='In-transit', index_col='Product')
CBM_df = pd.read_excel(data_file, sheet_name='Size', index_col='Product')
PriceAndCost_df = pd.read_excel(data_file, sheet_name='Price and cost', index_col='Product')
Shortage_df = pd.read_excel(data_file, sheet_name='Shortage', index_col='Product')
VendorProduct_df = pd.read_excel(data_file, sheet_name='Vendor-Product', index_col='Product') - 1
VendorCost_df = pd.read_excel(data_file, sheet_name='Vendor cost', index_col='Vendor')
Bound_df = pd.read_excel(data_file, sheet_name='Bounds', index_col='Product')
Conflict_df = pd.read_excel(data_file, sheet_name='Conflict', index_col='Conflict') - 1

N, M = Demand_df.shape
K = 3 #(express delivery, air freight, Ocean freight)
V = VendorCost_df.shape[0]
ContainerCap = 30 #Case 1 data
ContainerCost = 2750 #Case 1 data
# ContainerCap = 0.5 #Case 1 data
# ContainerCost = 1500 #Case 1 data

Demand_ij = Demand_df.to_numpy()
Init_i = Init_df.to_numpy().squeeze()
BuyCost_i = PriceAndCost_df['Purchasing cost'].to_numpy()
HoldCost_i = PriceAndCost_df['Holding cost'].to_numpy()
Transit_ij = InTransit_df.to_numpy()
ShipFixedCost_k = np.array([100, 80, 50]) #Case 1 data
ShipVarCost_ik = np.concatenate(( ShippingCost_df.to_numpy(), np.zeros((N,1)) ), axis=1)
CBM_i = CBM_df.to_numpy().squeeze()
LostSaleCost_i = Shortage_df['Lost sales'].to_numpy()
BackOrderCost_i = Shortage_df['Backorder'].to_numpy()
BackOrderProb_i = Shortage_df['Backorder percentage'].to_numpy()
VendorFixedCost_v = VendorCost_df.to_numpy().squeeze()
MinOrder_i = Bound_df.to_numpy().squeeze()
ConflictPair_alpha = Conflict_df.to_numpy()
ProductVendor_i = VendorProduct_df.to_numpy().squeeze()
M_big = np.sum(Demand_ij) + sum(Init_i)

In [5]:
#model
instance_name = 's1'
model = gb.Model(name=instance_name)

Academic license - for non-commercial use only - expires 2021-05-20
Using license file /Users/eason/gurobi.lic


In [6]:
#decision variable
x = model.addVars(N, M, K, vtype=GRB.INTEGER,name='x')
Abin = model.addVars(M, K, vtype=GRB.BINARY, name='Abin')
ContainerCnt = model.addVars(M, vtype=GRB.INTEGER, name='ContainerCnt')
StockLevel = model.addVars(N, M, vtype=GRB.CONTINUOUS, name='StockLevel')
Shortage = model.addVars(N, M, vtype=GRB.CONTINUOUS, name='Shortage')
Bbin = model.addVars(N, M, vtype=GRB.BINARY, name='Bbin')
Cbin = model.addVars(N, M, vtype=GRB.BINARY, name='Cbin')
Dbin = model.addVars(V, M, vtype=GRB.BINARY, name='Dbin')

In [7]:
#Expr
VolumeInOceanExpr_j = [
gb.LinExpr(
    quicksum(x[i,j,2] * CBM_i[i] for i in range(N))
)    
for j in range(M)]

BackOrderCntExpr_ij = [
    [
        gb.LinExpr( Shortage[i,j] * BackOrderProb_i[i] )
    for j in range(M)]
for i in range(N)]

LostSaleCntExpr_ij = [
    [
        gb.LinExpr( Shortage[i,j] * (1 - BackOrderProb_i[i]) )
    for j in range(M)]
for i in range(N)]

In [8]:
#obj function
TotalPurchaseCost = gb.LinExpr(
    quicksum(
        quicksum(
            quicksum(
                x[i,j,k] * BuyCost_i[i]
            for i in range(N))
        for j in range(M))
    for k in range(K))
)

TotalShipFixedCost = gb.LinExpr(
    quicksum(
        quicksum(
            Abin[j,k] * ShipFixedCost_k[k]
        for k in range(K))
    for j in range(M))
)

TotalShipVarCost = gb.LinExpr(
    quicksum(
        quicksum(
            quicksum(
                x[i,j,k] * ShipVarCost_ik[i,k]
            for k in range(K))
        for j in range(M))
    for i in range(N))
)

TotalHoldingCost = gb.LinExpr(
    quicksum(
        quicksum(
            StockLevel[i,j] * HoldCost_i[i]
        for i in range(N))
    for j in range(M))
)

TotalContainerCost = gb.LinExpr(
    quicksum(ContainerCnt[j] for j in range(M)) * ContainerCost
)

TotalBackOrderCost = gb.LinExpr(
    quicksum(
        quicksum(
            BackOrderCntExpr_ij[i][j] * BackOrderCost_i[i]
        for i in range(N))
    for j in range(M))
)

TotalLostSaleCost = gb.LinExpr(
    quicksum(
        quicksum(
            LostSaleCntExpr_ij[i][j] * LostSaleCost_i[i]
        for i in range(N))
    for j in range(M))
)

TotalVendorFixedCost = gb.LinExpr(
    quicksum(
        quicksum(
            Dbin[v,j] * VendorFixedCost_v[v]
        for v in range(V))
    for j in range(M))
)

In [9]:
#set obj function
model.setObjective(
    TotalPurchaseCost +
    TotalShipFixedCost + 
    TotalShipVarCost + 
    TotalHoldingCost + 
    TotalContainerCost + 
    TotalBackOrderCost + 
    TotalLostSaleCost +
    TotalVendorFixedCost
)

In [10]:
#Contrain

#let Abin to behave correctly
_ = model.addConstrs(
    quicksum(x[i,j,k] for i in range(N)) / M_big <= Abin[j,k]
for j in range(M)
for k in range(K))

#let ContainerCnt to behave correctly
_ = model.addConstrs(
    VolumeInOceanExpr_j[j] / ContainerCap <= ContainerCnt[j]
for j in range(M))

#let Stocklevle & Shortage behave correctly
_ = model.addConstrs(StockLevel[i,0] - Shortage[i,0] == Init_i[i] - Demand_ij[i][0] for i in range(N))
_ = model.addConstrs(StockLevel[i,1] - Shortage[i,1] == StockLevel[i,0] + x[i,0,0] + Transit_ij[i][1] - Demand_ij[i][1] - Shortage[i,0] * BackOrderProb_i[i] for i in range(N))
_ = model.addConstrs(StockLevel[i,2] - Shortage[i,2] == StockLevel[i,1] + x[i,1,0] + x[i,0,1] + Transit_ij[i][2] - Demand_ij[i][2] - Shortage[i,1] * BackOrderProb_i[i] for i in range(N))
_ = model.addConstrs(StockLevel[i,j] - Shortage[i,j] == StockLevel[i,j-1] + x[i,j-1,0] + x[i,j-2,1] + x[i,j-3,2] - Demand_ij[i][j] - Shortage[i,j-1] * BackOrderProb_i[i] for i in range(N) for j in range(3,M))

_ = model.addConstrs(StockLevel[i,j] <= M_big * (1-Bbin[i,j]) for i in range(N) for j in range(M))
_ = model.addConstrs(Shortage[i,j] <= M_big * Bbin[i,j] for i in range(N) for j in range(M))

#let Cbin behave correctly
_ = model.addConstrs(
    quicksum(x[i,j,k] for k in range(K)) / M_big <= Cbin[i,j]
for i in range(N)
for j in range(M))

#let Dbin behave correctly
_ = model.addConstrs(
    quicksum(
        quicksum(x[i,j,k] for k in range(K))
    for i in range(N) if ProductVendor_i[i] == v) / M_big <= Dbin[v,j]
for v in range(V)
for j in range(M))

#Minumin order Bound
_ = model.addConstrs(
    Cbin[i,j] * MinOrder_i[i] <= quicksum(x[i,j,k] for k in range(K))
for i in range(N)
for j in range(M))

#conflict
_ = model.addConstrs(
    Cbin[a,j] + Cbin[b,j] <= 1
for j in range(M)
for a, b in ConflictPair_alpha)

In [11]:
model.optimize()

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (mac64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 78026 rows, 109226 columns and 296960 nonzeros
Model fingerprint: 0x2bc0eeeb
Variable types: 31200 continuous, 78026 integer (31200 binary)
Coefficient statistics:
  Matrix range     [4e-07, 3e+06]
  Objective range  [1e+00, 7e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+06]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 16399 rows and 18841 columns
Presolve time: 1.41s
Presolved: 61627 rows, 90385 columns, 244982 nonzeros
Variable types: 26718 continuous, 63667 integer (19166 binary)
Found heuristic solution: objective 8.223949e+09

Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...

Extra simplex iterations after uncrush: 7327
Concurrent spin time: 0.08s

Solved with primal simplex

Root relaxati

In [12]:
os.makedirs('solution', exist_ok=True)
solution_file_path = path.join(work_dir, 'solution', 'q5_solution.xlsx')

In [13]:
print('Express delivery')
with pd.ExcelWriter(solution_file_path, engine='xlsxwriter') as writer:
    df1 = pd.DataFrame([[int(x[i,j,0].x) for j in range(M)] for i in range(N)], columns=range(M))
    display(df1)
    df1.to_excel(writer, sheet_name='Express delivery')
    print('Air frieght')
    df2 = pd.DataFrame([[int(x[i,j,1].x) for j in range(M)] for i in range(N)], columns=range(M))
    display(df2)
    df2.to_excel(writer, sheet_name='Air frieght')
    print('Ocean frieght')
    df3 = pd.DataFrame([[int(x[i,j,2].x) for j in range(M)] for i in range(N)], columns=range(M))
    display(df3)
    df3.to_excel(writer, sheet_name='Ocean frieght')
    print('StockLevel')
    df = pd.DataFrame([[StockLevel[i,j].x for j in range(M)] for i in range(N)], columns=range(M))
    display(df)
    df.to_excel(writer, sheet_name='StockLevel')
    print('Shortage')
    df = pd.DataFrame([[Shortage[i,j].x for j in range(M)] for i in range(N)], columns=range(M))
    display(df)
    df.to_excel(writer, sheet_name='Shortage')

Express delivery


,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
596,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
597,66,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
598,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Air frieght


,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
0,0,0,0,38,208,1,207,188,111,249,...,278,127,241,316,98,141,334,104,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,82,200,63,227,18,24,148,182,...,51,183,72,0,201,228,106,231,0,0
3,0,0,97,155,83,75,129,86,150,207,...,60,87,163,91,147,216,72,32,0,0
4,0,0,67,49,13,81,71,144,213,24,...,29,153,0,70,220,12,21,131,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,0,0,0,0,0,0,0,0,71,249,...,68,50,151,0,56,101,91,27,0,0
596,0,0,0,0,0,0,0,0,0,164,...,0,327,25,31,107,383,70,279,0,0
597,0,99,101,120,41,168,136,133,48,117,...,40,79,162,157,4,0,84,27,0,0
598,0,0,0,83,204,216,339,204,0,257,...,134,206,357,173,0,263,110,271,0,0


Ocean frieght


,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
0,0,0,0,0,0,0,1,0,0,1,...,0,0,1,0,0,1,0,0,0,0
1,0,7,38,136,118,58,121,78,206,166,...,109,60,89,74,244,131,155,0,0,0
2,0,0,0,0,0,0,1,0,0,1,...,0,0,1,0,0,1,0,0,0,0
3,0,0,0,0,0,0,1,0,0,1,...,0,0,1,0,0,1,0,0,0,0
4,0,0,0,0,0,0,1,0,0,0,...,0,0,1,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
596,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
597,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
598,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


StockLevel


,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
0,662.0,607.0,435.0,241.0,147.0,0.0,0.0,0.0,0.0,0.0,...,0.00000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
1,410.0,357.0,289.0,104.0,98.0,0.0,0.0,0.0,6.0,0.0,...,6.66386,0.0,0.0,6.835291,0.0,6.884704,0.0,0.0,0.0,0.0
2,346.0,167.0,166.0,117.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
3,208.0,258.0,180.0,49.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
4,365.0,303.0,220.0,130.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,768.0,694.0,654.0,613.0,455.0,237.0,216.0,200.0,25.0,25.0,...,0.00000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
596,1039.0,911.0,1062.0,1009.0,913.0,616.0,574.0,319.0,319.0,234.0,...,0.00000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
597,179.0,0.0,14.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
598,784.0,593.0,386.0,102.0,102.0,0.0,0.0,0.0,0.0,0.0,...,0.00000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0


Shortage


,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.33614,0.235298,0.0,0.164709,0.0,5.115296,6.580707,4.606495,5.224547
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
596,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
597,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
598,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000


In [14]:
print(f'TotalPurchaseCost    :{TotalPurchaseCost.getValue():.2f}')
print(f'TotalShipFixedCost   :{TotalShipFixedCost.getValue():.2f}')
print(f'TotalShipVarCost     :{TotalShipVarCost.getValue():.2f}')
print(f'TotalHoldingCost     :{TotalHoldingCost.getValue():.2f}')
print(f'TotalContainerCost   :{TotalContainerCost.getValue():.2f}')
print(f'TotalBackOrderCost   :{TotalBackOrderCost.getValue():.2f}')
print(f'TotalLostSaleCost    :{TotalLostSaleCost.getValue():.2f}')
print(f'TotalVendorFixedCost :{TotalVendorFixedCost.getValue():.2f}')
print(f'objective-value      :{model.ObjVal:.2f}')

TotalPurchaseCost    :877787417.01


NameError: name 'TotalShipFixedCost' is not defined